<a href="https://colab.research.google.com/github/JimCurryWang/Deep-Learning-Jot/blob/seq2seq-1/Transformer/2_Learning_Phrase_Representations_using_RNN_Encoder_Decoder_for_Statistical_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation


- Reference: [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)

-  Improving test perplexity while only using a single layer RNN in both the encoder and the decoder compared to first model in [1 - Sequence to Sequence Learning with Neural Networks](https://github.com/JimCurryWang/Deep-Learning-Jot/blob/seq2seq-1/Transformer/1_Sequence_to_Sequence_Learning_with_Neural_Networks.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1FGqD8Kv-rD3pbDpOTVT9I-WIUeLqAYum?usp=sharing)


# Introduction
- Learning Points
  - Re-use context vector (z) in decoder part for rnn component and final prediction linear layer output.
  - How to concatenate embedded and context together?
  - Why concatenate work in here? (a kind of Shortcut?)



In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

from pprint import pprint


SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Load Model & Prepare the data

In [8]:
# Load Spacy model
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

# tokenizer functions
# transformed sentence into a list of tokens
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    (SRC-source)
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    (TRG-target)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]


# torchtext's Field
SRC = Field(tokenize = tokenize_de, 
          init_token = '<sos>', 
          eos_token = '<eos>', 
          lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

# Download Multi30k dataset
train_data, valid_data, test_data = Multi30k.splits(
    exts = ('.de', '.en'), fields = (SRC, TRG))

In [9]:
# check the data by its length
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

# print out an single example, make sure the source sentence is reversed:
pprint(vars(train_data.examples[0]))

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
{'src': ['.',
         'büsche',
         'vieler',
         'nähe',
         'der',
         'in',
         'freien',
         'im',
         'sind',
         'männer',
         'weiße',
         'junge',
         'zwei'],
 'trg': ['two',
         'young',
         ',',
         'white',
         'males',
         'are',
         'outside',
         'near',
         'many',
         'bushes',
         '.']}


# Build Vocabulary

In [10]:
# use Filed object to build vocabulary
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


# Build iterators (BucketIterator*) for DataSet

In [11]:
# Build iterators (BucketIterator*) for DataSet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

# Building the Seq2Seq Model

## Encoder 

- Single-layer GRU

- The encoder is similar to the previous one, with the multi-layer LSTM swapped for a single-layer GRU. 

- As we only have a single layer, we also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. 

## Decoder

- The decoder is where the implementation differs significantly from the previous model and we alleviate some of the information compression.

- re-use context vector (z) in rnn component
  - re-use the same context vector returned by the encoder for every time-step in the decoder.
  
  - $$s_t = \text{DecoderGRU}(d(y_t), s_{t-1}, z)$$

- re-use contect vector in linear layer
  - Before, we predicted the next token, $\hat{y}_{t+1}$, with the linear layer, $f$, only using the top-layer decoder hidden state at that time-step, $s_t$, as $\hat{y}_{t+1}=f(s_t^L)$. 
  - Now, we also pass the embedding of current token, $d(y_t)$ and the context vector, $z$ to the linear layer.

  - $$\hat{y}_{t+1} = f(d(y_t), s_t, z)$$

-  Decoding single token per time-step (same as model-1)

  - The Decoder class does a single step of decoding. i.e. it ouputs single token per time-step. only decoding one token at a time, the input tokens will always have a sequence length of 1

-  Context vectors as fist input in Decoder (same as model-1)

  - the initial hidden and cell states to our decoder are our context vectors, which are the final hidden and cell states of our encoder from the same layer, i.e. $(s_0^l,c_0^l)=z^l=(h_T^l,c_T^l)$.

  - Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.


## Rethinking (Irresponsible Suggestion)
- The idea of re-using the context feature is somehow like adding short cut in U-net and ResNet?

In [58]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        '''
        '''
        super().__init__()

        self.hid_dim = hid_dim
        
        # no dropout as only one layer!
        self.embedding = nn.Embedding(
            num_embeddings=input_dim, embedding_dim=emb_dim
        )
        
        self.rnn = nn.GRU(input_size=emb_dim, hidden_size=hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        '''
        # src = [src len, batch size]
        # embedded = [src len, batch size, emb dim]
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]

        # outputs are always from the top hidden layer  
        '''
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
              
        return hidden


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.output_dim = output_dim

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(
            num_embeddings=output_dim, embedding_dim=emb_dim
        )
        
        self.rnn = nn.GRU(
            input_size= emb_dim + hid_dim, 
            hidden_size=hid_dim
        )
        
        self.fc_out = nn.Linear(
            in_features = emb_dim + hid_dim * 2, 
            out_features = output_dim
        )
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        '''
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hid dim]
        # context = [n layers * n directions, batch size, hid dim]
        
        # n layers and n directions in the decoder will both always be 1, therefore:
        # hidden = [1, batch size, hid dim]
        # context = [1, batch size, hid dim]
        '''
        # Decoding single token per time-step, 
        # so the input tokens will always have a sequence length of 1

        input = input.unsqueeze(0) 
        # -> input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input)) 
        # -> embedded = [1, batch size, emb dim]
        
        emb_con = torch.cat((embedded, context), dim = 2)
        # -> emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        # -> output = [seq len, batch size, hid dim * n directions]
        # -> hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        # -> output = [1, batch size, hid dim]
        # -> hidden = [1, batch size, hid dim]
        
        output = torch.cat(
            (embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
            dim = 1
        )
        # -> output = [batch size, emb dim + hid dim + contect dim]      
        #           = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        # -> prediction = [batch size, output dim]
        
        return prediction, hidden

# Seq2Seq

- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate (either the ground truth next token in the target sequence or the highest predicted next token)

In [59]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        '''
        # src = [src len, batch size]
        # trg = [trg len, batch size]
        
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        '''
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # ---- Encoder ----
        # last hidden state of the encoder is the context
        context = self.encoder(src)
        
        # ---- Decoder ----
        # context also used as the initial hidden state of the decoder
        hidden = context
        
        # first input to the decoder is the <sos> tokens
        # [<sos>, y1, y2, y3 ]
        # trg = [seq len, batch size], trg[0,:] -> first words for whole the batch
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            # insert input token embedding, previous hidden state and the context state
            # receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            # decide if we are going to use teacher forcing or not
            # teacher_force = [True, False]
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

# Training 

- Initialize our parameters. 
  - The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e.  N(0,0.01).

  - It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to  N(0,0.01).


- Optimizer & Loss Function
  - initiaize our optimizer.
  - initialize the loss function, making sure to ignore the loss on <pad> tokens.  

In [60]:
'''Note
# i.g. 
# TRG_PAD_IDX = 1 
# TRG.pad_token = '<pad>'
'''

INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5


def init_weights_uniform(m):
    '''initialize weights 
    with a uniform distribution(nn.init.uniform_) between -0.08 and +0.08
    '''
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

def init_weights_normal(m):
    '''initialize weights 
    with a normal distribution(nn.init.normal_) with a mean of 0 
    and a standard deviation of 0.01, 
    i.e. N(0,0.01) .
    '''
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)

def count_parameters(model):
    '''calculate the number of trainable parameters in the model
    '''
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# model
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

model.apply(init_weights_normal)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [62]:
# optimizer
optimizer = optim.Adam(model.parameters())
# loss function
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [63]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,220,293 trainable parameters


# Define Train & evaluate function

In [65]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        # --- forward --- 
        src = batch.src
        trg = batch.trg 
        output = model(src, trg)
        # -> trg = [trg len, batch size]
        # -> output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        # -> output = [(trg len - 1) * batch size, output dim]
        # -> trg = [(trg len - 1) * batch size]
        
        # loss computation
        loss = criterion(output, trg)

        # --- backward --- 
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent update step/adam step
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip) # gradient clipping
        optimizer.step()
        
        # loss computation records
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            # --- forward ---
            # turn off teacher forcing
            src = batch.src
            trg = batch.trg
            output = model(src, trg, 0)

            # -> trg = [trg len, batch size]
            # -> output = [trg len, batch size, output dim]

            # loss function
            # Allign the target and predicted_out
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            # -> output = [(trg len - 1) * batch size, output dim]
            # -> trg = [(trg len - 1) * batch size]
          
            loss = criterion(output, trg)

            # loss computation records
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

# Load Model & Evaluate on Test Set

### Relieving the information compression looks good!
- Just looking at the test loss, we get better performance than the previous model. 
- This is a pretty good sign that this model architecture is doing something right! 
- Relieving the information compression seems like the way forard.

### What can be improved next?
- Attention is all you need
  - In the next chapter we'll expand on this even further with `attention`.

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')